In [12]:
import numpy as np 
import pandas as pd
!pip install py7zr

from py7zr import unpack_7zarchive
import shutil

shutil.register_unpack_format('.7zip', '.7z', unpack_7zarchive)

shutil.unpack_archive('/kaggle/input/cifar-10/train.7z', '/kaggle/temp/')

RegistryError: . is already registered for ".7zip"

In [2]:
import torch
if torch.cuda.is_available():
    device=torch.device(type="cuda", index=0)
else:
    device=torch.device(type="cpu", index=0)

print(f'using device {device}')

using device cuda:0


In [3]:
labels=pd.read_csv("/kaggle/input/cifar-10/trainLabels.csv", header='infer')

classes=labels['label'].unique()

class_id = {}
i=0
for label in classes:
    class_id[label] = i
    i+=1

id_class = {}

for i in range(len(classes)):
    id_class[i] = classes[i]

In [4]:
print(class_id)
print(id_class)

{'frog': 0, 'truck': 1, 'deer': 2, 'automobile': 3, 'bird': 4, 'horse': 5, 'ship': 6, 'cat': 7, 'dog': 8, 'airplane': 9}
{0: 'frog', 1: 'truck', 2: 'deer', 3: 'automobile', 4: 'bird', 5: 'horse', 6: 'ship', 7: 'cat', 8: 'dog', 9: 'airplane'}


In [5]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Normalize, Resize, Compose

class TrainDataset(Dataset):
    def __init__(self, img, label):
        super().__init__()
        self.imgpath=img
        self.labelpath=label
        self.labels=pd.read_csv(label, header='infer')
        self.transform=Compose([Resize((224,224), antialias=True), Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
        
    def __len__(self):
        return self.labels.shape[0]
    
    def __getitem__(self,idx):
        finalpath=os.path.join(self.imgpath,str(idx+1))+'.png'
        img=read_image(finalpath)/255
        img=self.transform(img)
        label=self.labels.iloc[idx,1]
        label=class_id[label]
        return img,label

traindataset=TrainDataset('/kaggle/temp/train','/kaggle/input/cifar-10/trainLabels.csv')        
        
batch_size=64    
traindataloader=DataLoader(dataset=traindataset, batch_size=batch_size)

In [6]:
import torch.nn as nn
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights

model = mobilenet_v3_large(MobileNet_V3_Large_Weights.DEFAULT)
print(model)

class Prac5(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrainednet=mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT)
        self.pretrainednet.classifier=nn.Sequential(
            nn.Linear(in_features=960, out_features=1280, 
                   bias=True),nn.Hardswish(), 
            nn.Dropout(p=0.2, inplace=True), 
            nn.Linear(in_features=1280, out_features=10, 
                      bias=True)
        )
    def forward(self,x):
        x=self.pretrainednet(x)
        return x

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 126MB/s] 


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [7]:
def train(dataloader, model, loss, optimizer):
    model.train()
    track_loss=0
    num_correct=0
    num_param=0
    
    for i, (imgs, labels) in enumerate(dataloader):
        imgs=imgs.to(device)
        labels=labels.to(device)
        pred=model(imgs)
                    
        loss_e=loss(pred,labels)
        track_loss+=loss_e.item()
        num_correct+=(torch.argmax(pred,dim=1)==labels).type(torch.float).sum().item()
        
        running_loss=round(track_loss/(i+(imgs.shape[0]/batch_size)),2)
        running_acc=round((num_correct/((i*batch_size+imgs.shape[0])))*100,2)
        
        loss_e.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i%100==0:
            print("Batch:", i+1, "/",len(dataloader), "Running Loss:",running_loss, "Running Accuracy:",running_acc)
            
    epoch_loss=running_loss
    epoch_acc=running_acc
    return epoch_loss, epoch_acc

In [9]:
model=Prac5()
model=model.to(device)

for param in model.pretrainednet.features.parameters():
    param.requires_grad=False

loss=nn.CrossEntropyLoss()
lr=0.001
optimizer=torch.optim.Adam(params=model.parameters(), lr=lr)
n_epochs=5

for i in range(n_epochs):
    print("Epoch No:",i+1)
    epoch_loss, epoch_acc=train(traindataloader, model, loss, optimizer)
    print("Training:", "Epoch Loss:", epoch_loss, "Epoch Accuracy:", epoch_acc)
    print("--------------------------------------------------")

for param in model.pretrainednet.features.parameters():
    param.requires_grad=True

for i in range(n_epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_acc=train(traindataloader, model, loss, optimizer)
    print("Training:", "Epoch Loss:", train_epoch_loss, "Epoch Accuracy:", train_epoch_acc)
    print("--------------------------------------------------")

Epoch No: 1
Batch: 1 / 782 Running Loss: 2.31 Running Accuracy: 7.81
Batch: 101 / 782 Running Loss: 1.06 Running Accuracy: 63.97
Batch: 201 / 782 Running Loss: 0.92 Running Accuracy: 67.86
Batch: 301 / 782 Running Loss: 0.87 Running Accuracy: 69.77
Batch: 401 / 782 Running Loss: 0.83 Running Accuracy: 71.04
Batch: 501 / 782 Running Loss: 0.81 Running Accuracy: 71.61
Batch: 601 / 782 Running Loss: 0.8 Running Accuracy: 72.01
Batch: 701 / 782 Running Loss: 0.79 Running Accuracy: 72.41
Training: Epoch Loss: 0.78 Epoch Accuracy: 72.78
--------------------------------------------------
Epoch No: 2
Batch: 1 / 782 Running Loss: 0.63 Running Accuracy: 78.12
Batch: 101 / 782 Running Loss: 0.67 Running Accuracy: 76.76
Batch: 201 / 782 Running Loss: 0.64 Running Accuracy: 77.24
Batch: 301 / 782 Running Loss: 0.64 Running Accuracy: 77.57
Batch: 401 / 782 Running Loss: 0.64 Running Accuracy: 77.66
Batch: 501 / 782 Running Loss: 0.64 Running Accuracy: 77.7
Batch: 601 / 782 Running Loss: 0.64 Running

In [13]:
shutil.unpack_archive('/kaggle/input/cifar-10/test.7z', '/kaggle/temp/')

# shutil.register_unpack_format('7z')

shutil.register_unpack_format('.7zip', '.7z', unpack_7zarchive)

# shutil.unpack_archive('/kaggle/input/cifar-10/train.7z', '/kaggle/temp/')

RegistryError: . is already registered for ".7zip"

In [40]:
class TestDataset(Dataset):
    def __init__(self, imgpath):
        super().__init__()
        self.imgpath=imgpath
        _,_,self.files=next(os.walk(self.imgpath))
        self.length=len(self.files)
        self.transform=Compose([Resize((224,224), antialias=True), Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])        
    
    def __len__(self):
        return self.length
    
    def __getitem__(self,idx):
        finalpath=os.path.join(self.imgpath,str(idx+1))+'.png'
        img=read_image(finalpath)/255.0
        img=self.transform(img)
        return img

testdataset=TestDataset('/kaggle/temp/test/')
testdataloader=DataLoader(dataset=testdataset, batch_size=batch_size)

In [41]:
def eval(dataloader, model,loss_fn, path):
    model.eval()
    data=pd.read_csv(path)
    with torch.no_grad():
        for i, imgs in enumerate(dataloader):
            finalbatchpred=np.zeros(imgs.shape[0],dtype='object')
            imgs=imgs.to(device)
            pred=model(imgs)
            
            pred=torch.argmax(pred,dim=1).type(torch.int).cpu()
            for j,p in enumerate(pred):
                finalbatchpred[j]=id_class[p.item()]
            data.iloc[i*batch_size:i*batch_size+batch_size ,1]=finalbatchpred
    
    data.to_csv('submission.csv', index=False)
    data.head()
    
eval(testdataloader, model, loss, '/kaggle/input/cifar-10/sampleSubmission.csv')